# Object detection API training with PET dataset


這個 notebook 會教大家一步步的使用 tensorflow object detection API 來對 PET 這個資料集來進行訓練，使用原本 pre-train 在 COCO dataset 的 model 來進行 fine-tune。也可以參考[官方的說明](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_pets.md)

主要的步驟其實不多，詳述如下：
1. 下載 PET dataset，包含圖片與 label (annotations)
2. 將 label 與圖片轉檔成 tf_record 格式 (tensorflow 記錄檔案的格式，會把圖片跟標籤融合在一起)
3. 下載 pre-train 好的 model
4. 修改官方的 config 檔 (告訴程式碼資料放在哪裡、訓練的參數等細節都在這個 config 檔)
5. 設定好相關的環境變數
6. 硬 train 一發囉！！

**先跟大家說聲抱歉，目前 hub 的 tensorflow 版本不夠新，大家執行到 train.py 的時候是一定會遇到錯誤的，之後我們會再試著更新 tensorflow 版本**

### 1. 下載 PET 資料集

In [ ]:
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [ ]:
!tar zxvf images.tar.gz
!tar zxvf annotations.tar.gz

In [ ]:
!rm images.tar.gz
!rm annotations.tar.gz

### 2. 將 PET 的 label 與 images 轉檔為 tfrecord 的格式

tfrecord 是一個 tensorflow 專用的儲存檔案格式，以二進位記錄檔案，詳細可參考[這篇文章](http://blog.csdn.net/jinbaosite/article/details/75194226)。要使用 tensorflow API 要把 label 跟 Image 都轉換成 tfrecord 才能夠使用。tensorflow 已經有寫好常用資料集 label 轉換成 tfrecord 的程式，但如果要用自己標注好的資料集來訓練，則需要自己撰寫 label 轉 tfrecord 的程式，詳細可參考[教學](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md)

In [ ]:
%run object_detection/dataset_tools/create_pet_tf_record.py \
--label_map_path=object_detection/data/pet_label_map.pbtxt \
--data_dir= images/ \
--output_dir= data/

In [ ]:
# 將資料統一放在 data 這個資料夾中
!mv pet_train_with_masks.record object_detection/data/
!mv pet_val_with_masks.record object_detection/data/

### 3. 下載好 pre-train 的 model
更多的模型可參考 [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)，這邊我們下載 faster r-cnn，並使用 resnet 101 的 CNN 模型來提取特徵，這些模型都是已經在 COCO dataset 在 pre-train 過，基本上都已經有比較收斂的參數囉

In [ ]:
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz

In [ ]:
# 解壓縮
!tar zxvf faster_rcnn_resnet101_coco_11_06_2017.tar.gz

In [ ]:
# 將模型中的 checkpoint 統一放在 data 這個資料夾中
!cp faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.* object_detection/data/

### 4. 修改 model 的 config 檔
要提供 training data 的位置在哪裡、model 的 ckpt 等等細節給 config 檔，才能夠進行訓練，config 檔會長成[這個樣子](https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/faster_rcnn_resnet101_coco.config)，修改裡面的 **PATH_TO_BE_CONFIGURED**，改成我們統一放資料的那個資料夾路徑

In [ ]:
# 以這個 notebook 為例，要把路徑修改成剛剛統一放在 data 資料夾的路徑
"/home/jovyan/my_cv/session3&4/object_detection/data/"

In [ ]:
# 把修改好的 config 檔案統一放在 data 這個資料夾中
!cp object_detection/samples/configs/faster_rcnn_resnet101_pets.config \
object_detection/data/faster_rcnn_resnet101_pets.config

In [ ]:
# 給定我們修改好的 config 檔位置
PIPELINE_CONFIG = "object_detection/data/faster_rcnn_resnet101_pets.config"

# 訓練結果放置路徑
MY_MODEL_DIR="my_object_detect_model"

### 5. 設定相關環境變數
讓環境能夠看到 /models/research 與 /models/research/slim

若有問題可參考[連結](https://github.com/tensorflow/models/issues/1992)

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

### 6. 開 train 拉！！
給定 pipeline_config_path 與要儲存的 model 位置

In [ ]:
%run object_detection/train.py --logtostderr --pipeline_config_path=${PIPELINE_CONFIG}\
--train_dir=${MY_MODEL_DIR}/train